In [1]:
pip install pyomo

   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   -- ------------------------------------- 0.3/5.1 MB ? eta -:--:--
   -- ------------------------------------- 0.3/5.1 MB ? eta -:--:--
   -- ------------------------------------- 0.3/5.1 MB ? eta -:--:--
   -- ----------------------------

In [12]:
from pyomo.environ import *

# Create a Pyomo model
model = ConcreteModel()

# Parameters
C_internal = 750000  # Cost of internal maintenance per fault type
C_external_normal = 50000  # Cost of external maintenance per trip during normal season
C_external_high_demand = 150000  # Cost of external maintenance per trip during high-demand season
D_failure = 1000  # Cost of downtime per unaddressed fault
F_total = 10000  # Total faults across all types
F_type = [2000, 2500, 3000, 1500, 1000]  # Faults by type (5 types) # distribution is adjustable
p_external = 0.6  # Preventive success rate for external maintenance
U_total = 10000  # Total uptime (hours)
P_power = 150  # Price of power per MWh (in dollars)
B = 10000000  # Example budget

# Decision variables
model.x_internal = Var(range(5), domain=Binary)  # Binary decision for internal maintenance by fault type
model.x_external_normal = Var(domain=NonNegativeIntegers)  # External maintenance trips during normal season
model.x_external_high_demand = Var(domain=NonNegativeIntegers)  # External maintenance trips during high-demand season

# Unaddressed faults as a function of decision variables
def unaddressed_faults_rule(model):
    return F_total - sum(F_type[i] * model.x_internal[i] for i in range(5)) - \
           p_external * (model.x_external_normal + model.x_external_high_demand)
model.F_unaddressed = Expression(rule=unaddressed_faults_rule)

# Objective: Minimize total cost
def objective_rule(model):
    return (sum(C_internal * model.x_internal[i] for i in range(5)) +
            C_external_normal * model.x_external_normal +
            C_external_high_demand * model.x_external_high_demand +
            D_failure * model.F_unaddressed)
model.objective = Objective(rule=objective_rule, sense=minimize)

# Uptime constraint
def uptime_constraint_rule(model):
    uptime_remaining = U_total - (U_total * model.F_unaddressed / F_total)
    return P_power * uptime_remaining >= D_failure * model.F_unaddressed
model.uptime_constraint = Constraint(rule=uptime_constraint_rule)

# Budget constraint
def budget_constraint_rule(model):
    return (sum(C_internal * model.x_internal[i] for i in range(5)) +
            C_external_normal * model.x_external_normal +
            C_external_high_demand * model.x_external_high_demand) <= B
model.budget_constraint = Constraint(rule=budget_constraint_rule)

# Solve the model using Gurobi
solver = SolverFactory('gurobi')
results = solver.solve(model, tee=True)

# Display results
print("Optimal Solution:")
for i in range(5):
    print(f"x_internal[{i}] (fault type {i+1} handled internally): {model.x_internal[i].value}")
print(f"x_external_normal (trips during normal season): {model.x_external_normal.value}")
print(f"x_external_high_demand (trips during high-demand season): {model.x_external_high_demand.value}")
print(f"Total Cost: {model.objective()}")


Set parameter Username
Set parameter LicenseID to value 2599386
Academic license - for non-commercial use only - expires 2025-12-14
Read LP format model from file C:\Users\teren\AppData\Local\Temp\tmp0cavq40k.pyomo.lp
Reading time = 0.01 seconds
x1: 2 rows, 8 columns, 14 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 2 rows, 8 columns and 14 nonzeros
Model fingerprint: 0xa40d682b
Variable types: 1 continuous, 7 integer (5 binary)
Coefficient statistics:
  Matrix range     [7e+02, 3e+06]
  Objective range  [5e+04, 1e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+07, 1e+07]
Found heuristic solution: objective 3750000.0000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solut